In [1]:
#pip install gradio transformers sentence-transformers torch
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate  sentence-transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import requests
import json
from google.colab import userdata

from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util
import torch

import gradio as gr

2025-10-09 08:32:16.623969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759998737.028140      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759998737.159486      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create a cache folder on Drive
cache_dir = "/content/drive/MyDrive/huggingface"
os.makedirs(cache_dir, exist_ok=True)

# Set environment variable for Transformers cache
os.environ["TRANSFORMERS_CACHE"] = cache_dir

## HuggingFace models

In [3]:
# Sign in to HuggingFace Hub

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
if not hf_token:
    raise ValueError("Missing Hugging Face token in Colab secrets.")
#login(token=hf_token)
login(hf_token, add_to_git_credential=True)

Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [ ]:
from huggingface_hub import whoami
print(whoami())

In [5]:
import requests
requests.get("https://huggingface.co").status_code

200

In [19]:
# Embedding model
embed_model_name = "sentence-transformers/all-mpnet-base-v2"
# embed_model_name = "sentence-transformers/all-MiniLM-L12-v2"
# embed_model_name = "paraphrase-multilingual-mpnet-base-v2"
# embed_model_name = "hkunlp/instructor-large"  # requires special usage below

embedder = SentenceTransformer(embed_model_name, device="cuda" if torch.cuda.is_available() else "cpu")

# Rewriting model
llama_model = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
deepseek_model = 'deepseek-ai/deepseek-llm-7b-chat'
qwen2 = 'Qwen/Qwen2-7B-Instruct'

#selected model
model_id = llama_model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Creating Prompts

In [7]:
system_prompt = f"""You are an expert CV writing assistant.
Your goal is to generate tailored CV based on a given CV and job description.
Focus on matching key skills, tone, and phrasing, but keep the resume truthful.
Do not repeat the instructions."""

In [8]:
def complete_user_prompt(cv_text, job_description):
    user_prompt = f"""Please Generate a recruiter friendly CV for the following job:
      Job Description:
      {job_description}

      Resume Section:
      {cv_text}
      """
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

    return messages

## Accessing the Models

In [9]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU-Device:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("No GPU found.")

CUDA available: True
GPU-Device: Tesla T4


In [13]:
# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_quant_type= 'nf4'
)

# Load tokenizer + model
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    #cache_dir=cache_dir,
    quantization_config=quant_config,
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [14]:
def rewrite_cv(cv_text, job_description):
    try:
        messages = complete_user_prompt(cv_text, job_description)
        # Format messages correctly for chat template
        inputs = tokenizer.apply_chat_template(messages, return_tensors='pt').to('cuda')

        outputs = model.generate(
            inputs,
            max_new_tokens=2000,
            temperature=0.7,
            top_p=0.9,
            #streamer=streamer,  # remove if you don’t want live output
        )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        #del tokenizer, model, inputs, outputs #, streamer
        return generated_text.split(messages[-1]["content"])[-1].strip()

    except Exception as e:
        return f"Error during generation: {str(e)}"


In [20]:
def compute_similarity(cv_text, job_description):
    # Compute similarity score
    cv_emb = embedder.encode(cv_text, convert_to_tensor=True)
    job_emb = embedder.encode(job_description, convert_to_tensor=True)
    score = float(torch.nn.functional.cosine_similarity(cv_emb, job_emb, dim=0))

    return round(score * 100, 2)
    # Rewrite resume using LLaMA
    #rewritten = rewrite_cv(cv_text, job_description)

    #return f"**Semantic Match Score:** {similarity:.2f}\n\n**Rewritten Resume:**\n{rewritten}"

## Gradio UI

In [23]:
with gr.Blocks() as app:
    gr.Markdown("## 🧠 AI Resume Tailor — Powered by Hugging Face Transformers")

    with gr.Row():
        cv_input = gr.Textbox(label="Paste Your CV", lines=10, placeholder="Enter CV text...")
        job_input = gr.Textbox(label="Paste Job Description", lines=10, placeholder="Enter job description...")

    with gr.Row():
        output_resume = gr.Textbox(label="Tailored CV", lines=10)
        match_score = gr.Number(label="Match Score (%)")

    generate_btn = gr.Button("Tailor My CV")

    generate_btn.click(
        fn=lambda r, j: (rewrite_cv(r, j), compute_similarity(r, j)),
        inputs=[cv_input, job_input],
        outputs=[output_resume, match_score]
    )

app.launch(share=True)


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://cb701d6fbe47e3359a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]